Importando bibliotecas

In [ ]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt

Conectando ao MongoDB

In [ ]:
# Informações de conexão
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["dadosVivamente"]
collection = db["postsComPerguntasComInfos"]

Carregando os dados com filtro

In [ ]:
# Carregue os documentos da coleção que possuem nivel >= 2
filtro = {"nivel": {"$gte": 2}}
documentos = collection.find(filtro)

Convertendo para DataFrame

In [ ]:
df = pd.DataFrame(list(documentos))
df.head()

Transformando a coluna 'postCreated_time' para datetime

In [ ]:
df['postCreated_time'] = pd.to_datetime(df['postCreated_time'])
df.head()

Criando uma coluna 'data' apenas com o ano e o mês, descartando o dia

In [ ]:
df['data'] = df['postCreated_time'].dt.to_period('M').dt.to_timestamp()
df.head()

Agrupando por usuário e data, contando quantos posts cada usuário fez em cada período

In [ ]:
posts_grouped = df.groupby(['id_usuario', 'data']).size().reset_index(name='quantidade')

Criando o gráfico de linha para cada usuário

In [ ]:
for usuario in posts_grouped['id_usuario'].unique():
    df_usuario = posts_grouped[posts_grouped['id_usuario'] == usuario]

    # Ordenar os dados por data (se já não estiver)
    df_usuario = df_usuario.sort_values(by='data')

    # Configurar o gráfico de linha
    plt.figure(figsize=(18, 6))
    plt.plot(df_usuario['data'], df_usuario['quantidade'], marker='o', linestyle='-', color='blue')

    # Adicionar título e rótulos
    plt.title(f'Quantidade de Posts por Mês/Ano - Usuário: {usuario}')
    plt.xlabel('Mês/Ano')
    plt.ylabel('Quantidade de Posts')

    # Melhorar a legibilidade dos rótulos do eixo X, formatando como Mês/Ano
    # Exibir apenas um rótulo a cada 3 meses
    xticks = df_usuario['data'][::2]
    plt.xticks(xticks, xticks.dt.strftime('%m/%Y'), rotation=45, ha='right', fontsize=10)

    # Adicionar grid
    plt.grid(True, axis='y')

    # Ajustar o layout para evitar sobreposição
    plt.tight_layout()

    # Salvar o gráfico, se desejar
    # plt.savefig(f'grafico_posts_usuario_{usuario}.png')

    # Mostrar o gráfico
    plt.show()

    # Fechar a figura explicitamente para liberar memória
    plt.close()